Goals:
1. Create a corpus of all yt video descriptions for scam videos in the data.
2. vectorize this.
3. Use this to create a regressor capturing the degree to which a video description is "spam-speaky".
4.Use this bag-of-words model to generate values for a new variable, "spam-speakiness", for every observation in the dataset .
5. (optional) repeat this process with a dataset of spam-lookalike videos which warn AGAINST scams and spam and attempt to create another regressor, "virtue-speakiness".

In [1]:
import os
import json 
import pandas as pd
import pymongo
#from google.colab import drive
from pymongo import MongoClient
import socket
import urllib.request as urllib2
import pandas as pd
import numpy as np
import nltk  
import numpy as np  
import random  
import string

import urllib.request  
import re  



In [2]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)


db = client.videos_mdb
collection = db["videos"]  

In [3]:
#pull data from mongo (just one iv for now)
linVarList=["id","spam","description"]
df=pd.DataFrame(columns=linVarList)
row=[]
counter=0
data = collection.find({})
for item in data:
    for var in linVarList:
        if var in item:
            row.append(item[var])
        else:
            row.append(np.nan)
    df.loc[counter] = row
    counter+=1
    row=[]

In [4]:
spam=df.loc[df["spam"]==1]

In [5]:
rawDescriptions=[(item for item in spam["description"])]
corpus=""
for item in rawDescriptions:
    corpus=corpus.join(item)

In [6]:
 #nltk.download('punkt')

In [7]:
corpus = nltk.sent_tokenize(corpus)

In [8]:
for i in range(len(corpus )):
    corpus [i] = corpus [i].lower()
    corpus [i] = re.sub(r'\W',' ',corpus [i])
    corpus [i] = re.sub(r'\s+',' ',corpus [i])

In [11]:
wordfreq = {}
for sentence in corpus:
    tokens = nltk.word_tokenize(sentence)
    for token in tokens:
        if token not in wordfreq.keys():
            wordfreq[token] = 1
        else:
            wordfreq[token] += 1

In [12]:
import heapq
most_freq = heapq.nlargest(300, wordfreq, key=wordfreq.get)

In [14]:
boringWords=["with","that","this","will","your","there", "wesleyvirgin", "wesleymilliondollarvirgin","just", "they", "much","kaise","then", "than", "when", "here","these", "girl", "what","where","from", "wesley", "virgin", "also",  "about", "them", "which", "dave", "very"] #"girl" is removed b/c comes from name of big scam youtuber and leaving it in risks sexist codings

In [15]:
sentence_vectors = []
for sentence in corpus:
    sentence_tokens = nltk.word_tokenize(sentence)
    sent_vec = []
    for token in most_freq:
        if token in sentence_tokens:
            sent_vec.append(1)
        else:
            sent_vec.append(0)
    sentence_vectors.append(sent_vec)

In [16]:
sentence_vectors = np.asarray(sentence_vectors)

In [18]:
for item in sentence_vectors:
    print(item > 1)

[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False Fa

In [ ]:
#number of words in spamwords/ total number of words = regressor